In [6]:
import torch
import torch.nn as nn
from torchvision import models
alexnet = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
alexnet.eval()

def resnet18(pretrained : bool):
    encoder = models.__dict__['resnet18'](pretrained = pretrained)
    encoder.fc = nn.Identity()
    
    return encoder

resnet = resnet18(True)
resnet.eval()
# print(models.__dict__.keys())

Using cache found in /home/irmak/.cache/torch/hub/pytorch_vision_v0.10.0


mod_name: collections, name: OrderedDict
mod_name: torch._utils, name: _rebuild_parameter
mod_name: torch._utils, name: _rebuild_tensor_v2
mod_name: collections, name: OrderedDict
mod_name: torch._utils, name: _rebuild_parameter
mod_name: torch._utils, name: _rebuild_tensor_v2


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# Similar things for finding the stats but for image
class VisionSSLDataset(data.Dataset):
    def __init__(
        self,
        data_path,
        vision_stats=[], # Will have image means and stds
    ):
        super().__init__()
        self.roots = glob.glob(f'{data_path}/demonstration_*')
        self.roots = sorted(self.roots)

        self.data = load_data(self.roots, demos_to_use=[])

        vision_transforms = [
            T.Resize((480,640)),
            T.Lambda(crop_transform),
            T.ToTensor()
        ]
        if normalize:
            vision_transforms.append(
                T.Normalize(vision_stats[0], vision_stats[1])
            )
        self.vision_transform = T.Compose(vision_transforms)
        self.tactile_transform = T.Resize((16,16))
        self.tactile_only = tactile_only

    def __len__(self):
        return len(self.data['tactile']['indices'])

    def _get_image(self, demo_id, image_id):
        image_root = self.roots[demo_id]
        image_path = os.path.join(image_root, 'cam_1_rgb_images/frame_{}.png'.format(str(image_id).zfill(5)))
        img = self.vision_transform(loader(image_path))
        return torch.FloatTensor(img)

    def _get_tactile_image(self, tactile_values): # shape: 15,16,3
        tactile_image = torch.FloatTensor(tactile_values) 
        tactile_image = F.pad(tactile_image, (0,0,0,0,1,0), 'constant', 0) # pads it to 16,16,3 by concatenating 0z
        tactile_image = tactile_image.view(16,4,4,3)

        tactile_image = torch.concat([
            torch.concat([tactile_image[i*4+j] for j in range(4)], dim=0)
            for i in range(4)
        ], dim=1)
        tactile_image = torch.permute(tactile_image, (2,0,1))

        return self.tactile_transform(tactile_image)

    def __getitem__(self, index):
        demo_id, tactile_id = self.data['tactile']['indices'][index]
        _, image_id = self.data['image']['indices'][index]

        # Get the tactile image
        tactile_values = self.data['tactile']['values'][demo_id][tactile_id]
        tactile_image = self._get_tactile_image(tactile_values)

        # Get the camera image
        if self.tactile_only:
            return tactile_image, torch.empty(1) # Just return empty tensor if only tactile will be used

        image = self._get_image(demo_id, image_id)
        return tactile_image, image

    def getitem(self, index):
        return self.__getitem__(index) # NOTE: for debugging purposes